> Reminder: Use the ```cuopt-or``` kernel to run this notebook  

# GPU Acceleration Demo: VRPTW Optimization CPU vs GPU

This notebook demonstrates GPU acceleration for Vehicle Routing Problem with Time Windows (VRPTW) using OR-Tools (CPU) vs cuOpt (GPU) on Gehring & Homberger RC2 dataset.

**Objectives:**
- Compare CPU vs GPU performance on VRPTW optimization
- Measure solve-time speedups
- Verify solution feasibility and quality
- Demonstrate minimal migration effort (≤5 lines changed)

## Setup and Configuration

In [ ]:
import os
import fnmatch
import numpy as np
import pandas as pd
import zipfile
import traceback
import time

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

from cuopt.routing import DataModel, Solve, SolverSettings
import cudf
_ = cudf.Series([1,2,3]).sum() # Warmup
import cupy as cp

# Add utils to path
from utils.homberger_to_parquet import parse_homberger_file
from utils.timing import set_cpu_threads, run_timed

# Set reproducible seed
np.random.seed(123)

# Configure CPU threads for fair comparison
set_cpu_threads(12)

## Get and Prepare Dataset

In [ ]:
USE_SAMPLE = True  # Set to False for full dataset (1000 customers)
extract_dir = "data/homberger"

# Define data paths
if USE_SAMPLE:
    zip_file = "data/homberger_200_customer_instances.zip"
    instance_pattern = r"c2.*\.txt"  # C2 series, 200 customers
    print("Using SAMPLE dataset (C2 series - ~200 customers)")
else:
    zip_file = "data/homberger_1000_customer_instances.zip"
    instance_pattern = r"rc2.*\.txt"  # RC2 series, 1000 customers
    print("Using FULL dataset (RC2 series - ~1000 customers)")

# Create output directory
os.makedirs(extract_dir, exist_ok=True)

def extract_and_parse_homberger():
    """Extract and parse Homberger VRPTW instance from ZIP file."""
    if not os.path.exists(zip_file):
        raise FileNotFoundError(f"Data file not found: {zip_file}")

    print(f"📁 Extracting from: {os.path.basename(zip_file)}")

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Match files by pattern, regardless of extension
        if USE_SAMPLE:
            pattern = "C2_*"
        else:
            pattern = "RC2_*"
        matching_files = [f for f in zip_ref.namelist() if fnmatch.fnmatch(os.path.basename(f), pattern)]

        if not matching_files:
            # Fallback: use any file
            matching_files = zip_ref.namelist()

        instance_file = matching_files[0]
        print(f"📋 Using instance: {os.path.basename(instance_file)}")

        # Extract to temporary location
        temp_path = os.path.join(extract_dir, "temp_instance.txt")
        with zip_ref.open(instance_file) as source:
            with open(temp_path, 'wb') as target:
                target.write(source.read())

        try:
            customers_df, params = parse_homberger_file(temp_path)
            return customers_df, params
        finally:
            if os.path.exists(temp_path):
                os.remove(temp_path)

# Extract and parse the data
customers_df, vrptw_params = extract_and_parse_homberger()

print(f"\n📊 VRPTW Instance: {vrptw_params['instance']}")
print(f"Customers: {len(customers_df)}")
print(f"Vehicles: {vrptw_params['K']}")
print(f"Capacity: {vrptw_params['Q']}")
print(f"Depot: ({vrptw_params['depot']['x']}, {vrptw_params['depot']['y']})")
print(f"\n✅ Data loaded successfully")
print(f"Customer data schema:")
print(customers_df.info())

In [ ]:
def prepare_vrptw_data(customers_df, params):
    """Convert DataFrame to optimization-ready format"""
    
    # Add depot as customer 0
    depot = params['depot']
    depot_row = pd.DataFrame({
        'customer_id': [0],
        'x': [depot['x']],
        'y': [depot['y']],
        'demand': [0],
        'tw_start': [depot['tw_start']],
        'tw_end': [depot['tw_end']],
        'service_time': [depot['service_time']]
    })
    
    # Combine depot and customers
    all_locations = pd.concat([depot_row, customers_df], ignore_index=True)
    all_locations = all_locations.sort_values('customer_id').reset_index(drop=True)
    
    # Calculate distance matrix (Euclidean)
    n_locations = len(all_locations)
    distance_matrix = np.zeros((n_locations, n_locations))
    
    for i in range(n_locations):
        for j in range(n_locations):
            if i != j:
                dx = all_locations.iloc[i]['x'] - all_locations.iloc[j]['x']
                dy = all_locations.iloc[i]['y'] - all_locations.iloc[j]['y']
                distance_matrix[i][j] = int(np.sqrt(dx*dx + dy*dy))
    
    # Convert to lists for OR-Tools
    data = {
        'distance_matrix': distance_matrix.astype(int).tolist(),
        'demands': all_locations['demand'].tolist(),
        'time_windows': list(zip(all_locations['tw_start'], all_locations['tw_end'])),
        'service_times': all_locations['service_time'].tolist(),
        'num_vehicles': params['K'],
        'vehicle_capacity': params['Q'],
        'depot': 0
    }
    
    return data, all_locations

vrptw_data, locations_df = prepare_vrptw_data(customers_df, vrptw_params)

print(f"✅ VRPTW data prepared:")
print(f"Locations: {len(vrptw_data['distance_matrix']) - 1}")
print(f"Vehicles: {vrptw_data['num_vehicles']}")
print(f"Max distance: {np.max(vrptw_data['distance_matrix'])}")
print(f"Total demand: {sum(vrptw_data['demands'])}")

In [ ]:
CONFIG = {
    # Number of vehicles to use for both CPU and GPU
    'num_vehicles': vrptw_data['num_vehicles'] + 1,  # change to any int (e.g., 7)

    # Relax time windows by this percent on each side (0 = no relax).
    # Example: 20 means start -= 20% of (end-start), end += 20% of (end-start)
    'tw_relax_pct': -0.2,

    # Apply depot time window at vehicle start/end.
    # Keep False by default to match GPU if its API doesn’t enforce depot TW.
    'enforce_depot_tw': True,

    # OR Early Stopping Params
    'stall_secs': 15,
    'rel_eps': 0.001,
    'abs_eps': 0
}

## CPU Optimization - OR-Tools

In [ ]:
class EarlyStopOnStall:
    """
    Stop the Routing search when the incumbent hasn't improved by at least
    `rel_eps` for `stall_secs` seconds (wall clock) since the last improvement.
    """
    def __init__(self, routing_model: pywrapcp.RoutingModel,
                 stall_secs: float = 30.0,
                 rel_eps: float = 0.001,
                 abs_eps: float = 0):
        self._routing = routing_model
        self._stall_secs = float(stall_secs)
        self._rel_eps = float(rel_eps)
        self._abs_eps = int(abs_eps)
        self._best = None
        self._last_impr = time.time()

    def __call__(self):
        curr = int(self._routing.CostVar().Value())
        if self._best is None:
            self._best = curr
            self._last_impr = time.time()
            return

        rel_ok = curr <= self._best - max(int(self._best * self._rel_eps), 0)
        abs_ok = curr <= self._best - self._abs_eps
        if rel_ok or abs_ok:
            self._best = curr
            self._last_impr = time.time()
        else:
            if time.time() - self._last_impr >= self._stall_secs:
                # cooperative stop (ends current search cleanly)
                self._routing.solver().FinishCurrentSearch()


def solve_vrptw_ortools(data, use_relaxed_time_windows=False,
                        stall_secs=CONFIG['stall_secs'], rel_eps=CONFIG['rel_eps'], abs_eps=CONFIG['abs_eps']):
    """Solve VRPTW using OR-Tools with convergence-based early stop (no hard limits)."""

    num_vehicles = CONFIG['num_vehicles']

    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        num_vehicles,
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    # Distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Capacity constraint
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return int(data['demands'][from_node])

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        [data['vehicle_capacity']] * num_vehicles,
        True,  # start cumul to zero
        'Capacity'
    )

    # Time window constraint
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data['service_times'][from_node]
        travel_time = data['distance_matrix'][from_node][to_node]
        return int(service_time + travel_time)

    time_callback_index = routing.RegisterTransitCallback(time_callback)

    # Generous time horizon
    max_tw_end = max(tw[1] for tw in data['time_windows'])
    horizon = int(max_tw_end * 2)

    routing.AddDimension(
        time_callback_index,
        horizon,  # allow waiting time
        horizon,  # maximum time per vehicle
        False,    # don't force start cumul to zero
        'Time'
    )
    time_dimension = routing.GetDimensionOrDie('Time')

    # Symmetric TW relax helper
    relax_frac = float(CONFIG.get('tw_relax_pct', 0)) / 100.0
    def relaxed_tw(tw):
        start, end = int(tw[0]), int(tw[1])
        if relax_frac <= 0 or end <= start:
            return start, end
        width = end - start
        return max(0, int(start - relax_frac * width)), int(end + relax_frac * width)

    # Time windows (customers only; depot handled via start/end below)
    depot_idx = data['depot']
    depot_tw = data['time_windows'][depot_idx]
    for location_idx, tw in enumerate(data['time_windows']):
        if location_idx == depot_idx:
            continue  # skip depot here; handle as vehicle start/end if configured
        index = manager.NodeToIndex(location_idx)
        if index != -1:
            lo, hi = relaxed_tw(tw)
            time_dimension.CumulVar(index).SetRange(lo, hi)

    # Vehicle start/end constraints (optional depot TW enforcement)
    if CONFIG.get('enforce_depot_tw', False):
        for vehicle_id in range(num_vehicles):
            start_index = routing.Start(vehicle_id)
            end_index = routing.End(vehicle_id)
            time_dimension.CumulVar(start_index).SetRange(int(depot_tw[0]), int(depot_tw[1]))
            time_dimension.CumulVar(end_index).SetRange(int(depot_tw[0]), int(depot_tw[1]))
    # else: leave as default [0, horizon] to match GPU behavior if depot TW not enforced

    # Search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.log_search = False

    # Attach early-stop callback
    stall_cb = EarlyStopOnStall(routing, stall_secs=stall_secs, rel_eps=rel_eps, abs_eps=abs_eps)
    routing.AddAtSolutionCallback(stall_cb)

    # Solve
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        total_distance = 0
        total_served = 0
        total_utilization = 0
        used_vehicles = 0

        for vehicle_id in range(num_vehicles):
            index = routing.Start(vehicle_id)
            route = []
            route_distance = 0
            route_demand = 0

            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                route.append(node_index)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
                if node_index != data['depot']:
                    route_demand += data['demands'][node_index]

            route.append(manager.IndexToNode(index))  # depot at end

            if len(route) > 2:
                total_distance += route_distance
                total_served += len(route) - 2
                total_utilization += (route_demand / data['vehicle_capacity']
                                      if data['vehicle_capacity'] > 0 else 0)
                used_vehicles += 1

        avg_utilization = total_utilization / used_vehicles if used_vehicles > 0 else 0.0

        return {
            'feasible': True,
            'objective': total_distance,
            'num_routes': used_vehicles,
            'customers_served': total_served,
            'avg_utilization': avg_utilization
        }
    else:
        return {'feasible': False}

In [ ]:
print("Starting CPU Solve...")
cpu_solution, cpu_solve_time = run_timed(
    "CPU Solve (OR-Tools)", 
    lambda: solve_vrptw_ortools(vrptw_data),
    use_gpu=False
)
print(f"Feasible: {cpu_solution['feasible']}")

if cpu_solution['feasible']:
    print(f"\n📊 CPU Results (OR-Tools):")
    print(f"Routes: {cpu_solution['num_routes']}")
    print(f"Customers served: {cpu_solution['customers_served']}/{len(customers_df)}")
    print(f"Total route distance: {cpu_solution['objective']}")
    print(f"Average vehicle utilization: {cpu_solution['avg_utilization']:.2f}")
    print(f"Solve time: {cpu_solve_time:.3f}s")

    print(f"\n✅ CPU optimization completed")

## GPU Optimization - cuOpt

In [ ]:
def solve_vrptw_cuopt(data):
    """Solve VRPTW using NVIDIA cuOpt with correct route extraction"""
    
    n_locations = len(data['distance_matrix'])
    n_orders = n_locations - 1
    n_vehicles = CONFIG['num_vehicles']
        
    try:
        # Create cuOpt DataModel
        data_model = DataModel(n_locations, n_vehicles, n_orders)

        # Set distance/cost matrix using cuDF with proper dtype
        distance_matrix_cudf = cudf.DataFrame(data['distance_matrix'], dtype='int32')
        data_model.add_cost_matrix(distance_matrix_cudf)

        # Set vehicle capacities and demands using cuDF with proper dtypes
        demands_cudf_orders = cudf.Series(data['demands'][1:], dtype='int32')  # exclude depot
        vehicle_capacities_cudf = cudf.Series([data['vehicle_capacity']] * n_vehicles, dtype='int32')
        data_model.add_capacity_dimension("demand", demands_cudf_orders, vehicle_capacities_cudf)

        # Set order locations - exclude depot from orders (GPU-native)
        order_indices = cudf.Series(cp.arange(1, n_locations, dtype=cp.int32))
        data_model.set_order_locations(order_indices)

        # Symmetric TW relax helper (on host, small; transferred as device arrays once)
        relax_frac = float(CONFIG.get('tw_relax_pct', 0)) / 100.0
        def relaxed_tw(tw):
            start, end = int(tw[0]), int(tw[1])
            if relax_frac <= 0 or end <= start:
                return start, end
            width = end - start
            return max(0, int(start - relax_frac * width)), int(end + relax_frac * width)

        # Set order time windows (exclude depot)
        earliest_times = []
        latest_times = []
        for tw in data['time_windows'][1:]:
            lo, hi = relaxed_tw(tw)
            earliest_times.append(int(lo))
            latest_times.append(int(hi))
        earliest_times = cudf.Series(earliest_times, dtype='int32')
        latest_times = cudf.Series(latest_times, dtype='int32')
        data_model.set_order_time_windows(earliest_times, latest_times)

        # Set service times using cuDF with proper dtype (exclude depot)
        service_times = cudf.Series([int(st) for st in data['service_times'][1:]], dtype='int32')
        data_model.set_order_service_times(service_times)

        # Set vehicle start/end locations using cuDF
        depot_idx = int(data['depot'])
        vehicle_start_locs = cudf.Series([depot_idx] * n_vehicles, dtype='int32')
        vehicle_end_locs = cudf.Series([depot_idx] * n_vehicles, dtype='int32')
        data_model.set_vehicle_locations(vehicle_start_locs, vehicle_end_locs)

        # Optionally enforce depot TW at vehicle start/end if API available
        if CONFIG.get('enforce_depot_tw', False):
            depot_tw = data['time_windows'][depot_idx]
            start_t, end_t = int(depot_tw[0]), int(depot_tw[1])
            if hasattr(data_model, 'set_vehicle_time_windows'):
                starts = cudf.Series([start_t] * n_vehicles, dtype='int32')
                ends = cudf.Series([end_t] * n_vehicles, dtype='int32')
                data_model.set_vehicle_time_windows(starts, ends)

        # Configure solver settings (keep your limit as-is)
        solver_settings = SolverSettings()
        solver_settings.set_time_limit(600)

        routing_solution = Solve(data_model, solver_settings)

        # Objective as scalar (tiny host copy)
        if hasattr(routing_solution, 'get_total_objective'):
            total_cost = float(routing_solution.get_total_objective())
        elif hasattr(routing_solution, 'total_objective_value'):
            total_cost = float(routing_solution.total_objective_value)
            print("Use this one")
        else:
            total_cost = 0.0

        route_df = routing_solution.get_route() #route_df is already cuDF

        # Compute customers served and avg utilization entirely on device
        df_visits = route_df[route_df['location'] != depot_idx]

        # customers_served: bring back only scalar
        customers_served = int(len(df_visits))

        # avg_utilization: gather demands by location on device, group by truck
        # Build device-side demands vector (full locations including depot)
        idx = (df_visits['location'] - 1).astype('int32')  # depot filtered out, so >= 0
        df_visits['demand'] = demands_cudf_orders.take(idx)

        per_truck = df_visits.groupby('truck_id')['demand'].sum()
        used_vehicles = int(len(per_truck))
        cap = max(1, int(data['vehicle_capacity']))
        if used_vehicles == 0:
            avg_utilization = 0.0
        else:
            util_series = (per_truck.astype('float32') / float(cap)).fillna(0)
            avg_utilization = float(util_series.mean())

        return {
            'feasible': True,
            'objective': int(total_cost) if total_cost else 0,
            'num_routes': used_vehicles,
            'customers_served': customers_served,
            'avg_utilization': avg_utilization
        }

    except Exception as solve_error:
        print(f"❌ cuOpt solve failed with error: {solve_error}")
        print("\n🔍 Full stack trace:")
        traceback.print_exc()
        return {'feasible': False}

In [ ]:
print("🚀 Starting GPU Solve...")
gpu_solution, gpu_solve_time = run_timed(
    "GPU Solve (cuOpt)", 
    lambda: solve_vrptw_cuopt(vrptw_data),
    use_gpu=True
)
print(f"Feasible: {gpu_solution['feasible']}")

if gpu_solution['feasible']:
    print(f"\n📊 GPU Results (cuOpt):")
    print(f"Feasible: {gpu_solution['feasible']}")
    print(f"Total route distance: {gpu_solution['objective']}")
    print(f"Routes: {gpu_solution['num_routes']}")
    print(f"Customers served: {gpu_solution['customers_served']}/{len(customers_df)}")
    print(f"Average vehicle utilization: {gpu_solution['avg_utilization']:.2f}")
    print(f"Solve time: {gpu_solve_time:.3f}s")

    print(f"\n✅ GPU optimization completed")

## Performance Comparison and Analysis

In [ ]:
# Calculate solve time speedup
if gpu_solve_time > 0:
    solve_speedup = cpu_solve_time / gpu_solve_time
else:
    solve_speedup = float('inf')

# Calculate solution quality metrics
if cpu_solution['feasible'] and gpu_solution['feasible']:
    if cpu_solution['objective'] > 0:
        objective_improvement = (cpu_solution['objective'] - gpu_solution['objective']) / cpu_solution['objective'] * 100
    else:
        objective_improvement = 0.0
else:
    objective_improvement = 0.0

# Create comparison table
comparison_data = [
    {
        'Metric': 'Solve Time (s)',
        'CPU (OR-Tools)': f"{cpu_solve_time:.3f}",
        'GPU (cuOpt)': f"{gpu_solve_time:.3f}",
        'Speedup/Improvement': f"{solve_speedup:.1f}x"
    },
    {
        'Metric': 'Total Vehicle Distance',
        'CPU (OR-Tools)': f"{cpu_solution['objective']}",
        'GPU (cuOpt)': f"{gpu_solution['objective']}",
        'Speedup/Improvement': f"{objective_improvement:+.1f}%" if abs(objective_improvement) > 0.1 else "Same"
    },
    {
        'Metric': 'Routes Used',
        'CPU (OR-Tools)': f"{cpu_solution['num_routes']}",
        'GPU (cuOpt)': f"{gpu_solution['num_routes']}",
        'Speedup/Improvement': f"{cpu_solution['num_routes'] - gpu_solution['num_routes']:+d}" if cpu_solution['num_routes'] != gpu_solution['num_routes'] else 'Same'
    },
    {
        'Metric': 'Customers Served',
        'CPU (OR-Tools)': f"{cpu_solution['customers_served']}/{len(customers_df)}",
        'GPU (cuOpt)': f"{gpu_solution['customers_served']}/{len(customers_df)}",
        'Speedup/Improvement': 'Same' if cpu_solution['customers_served'] == gpu_solution['customers_served'] else 'Different'
    },
    {
        'Metric': 'Avg. Vehicle Utilization',
        'CPU (OR-Tools)': f"{cpu_solution['avg_utilization']:.2f}",
        'GPU (cuOpt)': f"{gpu_solution['avg_utilization']:.2f}",
        'Speedup/Improvement': 'Same' if cpu_solution['avg_utilization'] == gpu_solution['avg_utilization'] else 'Different'
    }
]

comparison_df = pd.DataFrame(comparison_data)
print("⚡ VRPTW Optimization Comparison:")
print(comparison_df.to_string(index=False))

# Solution quality check
solution_quality_ok = (
    cpu_solution['feasible'] == gpu_solution['feasible'] and
    cpu_solution['customers_served'] == gpu_solution['customers_served']
)